In [3]:
import os

In [4]:
%pwd

'c:\\Users\\NGASSAM\\Desktop\\NONO\\End-to-end-Machine-Learning-Project-with-Mlflow\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\NGASSAM\\Desktop\\NONO\\End-to-end-Machine-Learning-Project-with-Mlflow'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [45]:
from mlProject.constants import *

In [44]:
from dataclasses import dataclass
from pathlib import Path
from mlProject.utils.common import read_yaml, create_directories

In [43]:
# Définir les chemins par défaut
CONFIG_FILE_PATH = "config/config.yaml"
PARAMS_FILE_PATH = "params.yaml"
SCHEMA_FILE_PATH = "schema.yaml"

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        """Charge les fichiers YAML de configuration et crée les répertoires nécessaires."""
        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))
        self.schema = read_yaml(Path(schema_filepath))

        # Créer le répertoire racine des artifacts
        create_directories([self.config["artifacts_root"]])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Récupère la configuration pour l'ingestion des données."""
        config = self.config["data_ingestion"]  # Dictionnaire contenant la config d'ingestion

        # Créer les répertoires nécessaires
        create_directories([Path(config["root_dir"])])

        # Retourner l'objet DataIngestionConfig
        return DataIngestionConfig(
            root_dir=Path(config["root_dir"]),
            source_url=config["source_URL"],  # Assurez-vous que la clé est bien "source_URL" dans config.yaml
            local_data_file=Path(config["local_data_file"]),
            unzip_dir=Path(config["unzip_dir"])
        )

import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        """Initialise l'ingestion des données avec la configuration fournie."""
        self.config = config

    def download_file(self):
        """Télécharge le fichier si celui-ci n'existe pas déjà."""
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} téléchargé avec succès !\nInfos : \n{headers}")
        else:
            logger.info(f"Le fichier existe déjà, taille : {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        """Décompresse le fichier ZIP téléchargé."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extraction réussie dans {unzip_path}")

# Exécution du script
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-02-19 16:50:25,209: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-19 16:50:25,213: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-19 16:50:25,216: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-19 16:50:25,218: INFO: common: created directory at: artifacts]
[2025-02-19 16:50:25,220: INFO: common: created directory at: artifacts\data_ingestion]
[2025-02-19 16:50:26,085: INFO: 3278250095: artifacts\data_ingestion\data.zip téléchargé avec succès !
Infos : 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD6A:34B3AA:35D0C:375B6:67B5FDC4
Accept-Ranges: bytes
Date: Wed, 19